# project 2

## set up

### step 0

In [12]:
# step 0
# I use macbook. I have created a folder for this project, then I create a virtal environment for 
# the python code.
# If you also use Macbook, here is the command I have used
# cd /path/to/your_project
# python3 -m venv .venv
# source .venv/bin/activate

### step 1

In [ ]:
# step 1: upgrade pip, skip this step if you already upgraded pip
pip install --upgrade pip

### step 2

In [ ]:
# step 2: install relevant packages, kip this step if you already have insatlled them
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install scikit-image

## my solution

### import packages

In [1]:
# import the relevant packages
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color 
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# global variables which is speicfied in the task description
n0 = 28 # input dimention
n1 = 64 # hidden layer 1
n2 = 32 # hidden layer 2
n3 = 3  # output dimention

### load data

In [3]:
# 1. Load MNIST digits
digits = load_digits()
X = digits.data  # shape: (1797, 64) but we use softplus MLP，OK
y = digits.target

# 2. Filter classes (1,5,7)
mask = np.isin(y, [1, 5, 7])
X = X[mask]
y = y[mask]

# map labels to {0,1,2}
label_map = {1:0, 5:1, 7:2}
y = np.array([label_map[v] for v in y])

# 3. Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Standardize
# z = (x - u)/sigma
# u: mean, sigma:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
print(X.shape)
print(y.shape)

(543, 64)
(543,)


In [ ]:
# wie habe y samples, K categories
# each row means to which category does the sample belong to
def one_hot(y, K=3):
    N = len(y)
    Y = np.zeros((N, K))
    Y[np.arange(N), y] = 1
    return Y

In [ ]:
# Test
# the example below shows how is the input and output 
# when you use one_hot function
y = np.array([2,1,1,1])
Y = np.zeros((4,3))
print(Y)
result = one_hot(y)
print(result)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [8]:
def softplus(x):
    return np.log1p(np.exp(-np.abs(x))) + np.maximum(x, 0)

def d_softplus(x):
    return 1 / (1 + np.exp(-x))   # sigmoid

In [10]:
np.exp(0)

np.float64(1.0)

In [ ]:
np.log(100) # base is e

np.float64(4.605170185988092)

In [12]:
np.log10(100)

np.float64(2.0)

In [ ]:
np.log2(4) # base 2

np.float64(2.0)

In [17]:
np.log1p(1.7) # base e, it calculates log(1+x)

np.float64(0.9932517730102833)

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    s = sigmoid(x)
    return s * (s - 1)

In [ ]:
def forward():
    pass